In [1]:
import pandas as pd
import os
import json
from pandas.io.json import json_normalize
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)
import torchvision
import torch.nn as nn
# from torchsummary import summary
import tokenization

import argparse
import collections
import logging
import json
import math
import os
import random
import six
from tqdm import tqdm_notebook as tqdm
from IPython.display import HTML, display

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from torchtext import data,datasets
from torchtext.vocab import GloVe,FastText,CharNGram
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import sys
import torchtext.data.dataset

import tokenization
from modeling import BertConfig, BertForMaskedLanguageModelling
from optimization import BERTAdam
from masked_language_model import notqdm, convert_tokens_to_features, LMProcessor, predict_masked_words, predict_next_words, improve_words_recursive

from sklearn.model_selection import train_test_split

import re
import csv

print(os.listdir())

['.circleci', '.git', '.github', '.gitignore', '.ipynb_checkpoints', 'askreddit_data.json', 'Bert ADL Project.ipynb', 'CONTRIBUTING.md', 'convert_tf_checkpoint_to_pytorch.py', 'data', 'dataset.py', 'detokenization.py', 'docker', 'docs', 'examples', 'file_0.csv', 'hubconf.py', 'hubconfs', 'LICENSE', 'MANIFEST.in', 'masked_language_model.py', 'modeling.py', 'notebooks', 'optimization.py', 'outputs.html', 'pytorch_pretrained_bert', 'README.md', 'requirements.txt', 'samples', 'setup.py', 'tests', 'test_threads.txt', 'tokenization.py', 'train_threads.txt', 'uncased_askreddit_data.tsv', '__init__.py', '__pycache__']


In [2]:
data = json.load(open('./askreddit_data.json', 'r'))

# ask_reddit_data =  pd.read_json('./askreddit_data.json')
# ask_reddit_data = json_normalize(data)
uncased_ask_reddit_data = pd.read_csv('./uncased_askreddit_data.tsv', sep='\t', header=0)

In [71]:
uncased_ask_reddit_data.head()
# uncased_ask_reddit_data['submission id'] == 'bn3bpz'

,submission score,comment score,submission id,comment id,title (string #1),comment (string#2)
0,80513,25495,bn3bpz,en27b8v,would you support a mandatory environmental se...,"not mandatory, but there was a voluntary progr..."
1,80513,4686,bn3bpz,en2anug,would you support a mandatory environmental se...,i feel like instead of forcing people to do so...
2,80513,2041,bn3bpz,en254lj,would you support a mandatory environmental se...,i wouldn't make it mandatory but maybe optiona...
3,80513,608,bn3bpz,en2gacy,would you support a mandatory environmental se...,"we already have americorps, where you serve fo..."
4,80513,310,bn3bpz,en2p5ro,would you support a mandatory environmental se...,i feel like forcing people to do it will just ...


In [5]:
array_id = uncased_ask_reddit_data['submission id']

set_of_ids = set(array_id)

print(set_of_ids, len(set_of_ids))

file_train, file_test = train_test_split(list(set_of_ids), random_state=42, test_size=0.10)
print(file_test, len(file_test))
# print(file_train, len(file_train))
# with open('train_threads.txt', 'a+') as file_train_ids:
#     for elem in file_train:
#         file_train_ids.write(elem + '\n')

# # with open('test_threads.txt', 'a+') as file_test_ids:
#     for elem in file_test:
#         file_test_ids.write(elem + '\n')

{'bncqlb', 'bq7pbb', 'btexev', 'bls9ab', 'bpr8xn', 'bri3ja', 'bojhpr', 'bqltzq', 'blrjnj', 'bqhfq9', 'bskplh', 'bkxl3u', 'bmhq67', 'bty7a5', 'bkjqjj', 'bmt72c', 'bubm6g', 'bn3ab7', 'bvgvkh', 'bq2sd5', 'btfu8k', 'buz6lq', 'br87g0', 'bovsdb', 'bu1s5i', 'bs3th0', 'blna2q', 'brzuwa', 'bqx0lw', 'bvr285', 'bk8ypo', 'boxktx', 'br19bh', 'boi2wy', 'bogywr', 'bo3owh', 'blzj3m', 'bo6854', 'bmhwr8', 'bobtwf', 'bmjk7w', 'brlti4', 'buso57', 'bv51tj', 'bt3zc7', 'bsg50t', 'bstmp7', 'blf1l9', 'bvdaci', 'bmyv30', 'bm8dfx', 'bnlib2', 'bn3bpz', 'bp9xm7', 'bp182w', 'bkh4qb', 'bpw8r2', 'bniexw', 'bljq8j', 'bs6tqc', 'bro41c', 'bsahi8', 'btm0yy', 'bue7cv', 'bsomsy', 'bqc3ib', 'brayzq', 'bu766u', 'bkkoq2', 'bntob1', 'budsip', 'bqg2xi', 'bltmmq', 'buluna', 'bmac4g', 'bv6o78', 'btb5si', 'bndpre', 'buunu0', 'bkckiw', 'bkseqj', 'bugoyw', 'bl2scw', 'bmlzqs', 'bnqk9j', 'bkw3yd', 'bvkru1', 'blcbrl', 'bk6w6e', 'bquclv', 'bttd2p'} 91
['bmjk7w', 'br87g0', 'bkh4qb', 'bk6w6e', 'bncqlb', 'blna2q', 'bobtwf', 'brayzq', 'bskp

In [6]:
# uncased_ask_reddit_data['submission id'] == 'bn3bpz'
# uncased_ask_reddit_data.loc[uncased_ask_reddit_data['submission id'] == 'bn3bpz']
train_df = [] #pd.DataFrame(columns=['submission score	comment score', 'submission id', 'comment id', 'title (string #1)', 'comment (string#2)'])
test_df = []

with open('train_threads.txt', 'r') as trt:
    for elem in trt.readlines():
#         print(elem.strip())
#         print(uncased_ask_reddit_data.loc[uncased_ask_reddit_data['submission id'] == elem.strip()])
        train_df.append(uncased_ask_reddit_data.loc[uncased_ask_reddit_data['submission id'] == elem.strip()])
        

In [7]:
with open('test_threads.txt', 'r') as tst:
    for elem in tst.readlines():
#         print(elem.strip())
#         print(uncased_ask_reddit_data.loc[uncased_ask_reddit_data['submission id'] == elem.strip()])
        test_df.append(uncased_ask_reddit_data.loc[uncased_ask_reddit_data['submission id'] == elem.strip()])

In [8]:
# print(train_df[:3])
# train_df_parsed = pd.DataFrame(train_df[0])
# train_df_parsed.to_csv(path_or_buf='./file_0.csv', sep='\t')

for i in range(len(train_df)):
    train_df_parsed = pd.DataFrame(train_df[i])
    train_df_parsed.to_csv(path_or_buf='./data/train/file_'+str(i)+'.csv', sep='\t')


In [9]:
for i in range(len(test_df)):
    test_df_parsed = pd.DataFrame(train_df[i])
    test_df_parsed.to_csv(path_or_buf='./data/test/file_'+str(i)+'.csv', sep='\t')


In [18]:
data_dict = dict()
data_dict['submission score'] = []
data_dict['comment score'] = []
data_dict['submission id'] = []
data_dict['comment id'] = []
data_dict['title (string #1)'] = []
data_dict['comment (string#2)'] = []

for i in range(81):
#     print(i, './data/train/file_'+str(i)+'.csv')
    with open('./data/train/file_'+str(i)+'.csv') as tsvfile:
        reader = csv.DictReader(tsvfile, dialect='excel-tab')
    #     next(reader)
    #     df = pd.DataFrame(reader.data, columns=readr.columns)
    #     print(df.head())
        for row in reader:
    #         print(row)
    #         data_dict += row
            data_dict['submission score'].append(row['submission score'])
            data_dict['comment score'].append(row['comment score'])
            data_dict['submission id'].append(row['submission id'])
            data_dict['comment id'].append(row['comment id'])
            data_dict['title (string #1)'].append(row['title (string #1)'])
            data_dict['comment (string#2)'].append(row['comment (string#2)'])
        
# print(data_dict.values())

data_df_train = pd.DataFrame(data=data_dict)

print(data_df_train)

     submission score comment score submission id comment id  \
0               29117          6431        bncqlb    en4pxcp   
1               29117         24298        bncqlb    en4hp28   
2               29117         15415        bncqlb    en4kqq9   
3               29117         14389        bncqlb    en4f7vh   
4               29117          5357        bncqlb    en4m0gm   
5               29117         16081        bncqlb    en4hvq1   
6               29117         25806        bncqlb    en4n6hx   
7               29117          4892        bncqlb    en4izto   
8               29117         12290        bncqlb    en4hijc   
9               29117         17502        bncqlb    en4lisi   
10              29117         19126        bncqlb    en4et42   
11              29117         16508        bncqlb    en4je3s   
12              29117          4241        bncqlb    en4fcs2   
13              29117          6952        bncqlb    en4ip4o   
14              29117          2167     

[7204 rows x 6 columns]


In [17]:
data_dict = dict()
data_dict['submission score'] = []
data_dict['comment score'] = []
data_dict['submission id'] = []
data_dict['comment id'] = []
data_dict['title (string #1)'] = []
data_dict['comment (string#2)'] = []

for i in range(10):
#     print(i, './data/train/file_'+str(i)+'.csv')
    with open('./data/test/file_'+str(i)+'.csv') as tsvfile:
        reader = csv.DictReader(tsvfile, dialect='excel-tab')
    #     next(reader)
    #     df = pd.DataFrame(reader.data, columns=readr.columns)
    #     print(df.head())
        for row in reader:
    #         print(row)
    #         data_dict += row
            data_dict['submission score'].append(row['submission score'])
            data_dict['comment score'].append(row['comment score'])
            data_dict['submission id'].append(row['submission id'])
            data_dict['comment id'].append(row['comment id'])
            data_dict['title (string #1)'].append(row['title (string #1)'])
            data_dict['comment (string#2)'].append(row['comment (string#2)'])
        
# print(data_dict.values())

data_df_test = pd.DataFrame(data=data_dict)

print(data_df_test)

    submission score comment score submission id comment id  \
0              29117          6431        bncqlb    en4pxcp   
1              29117         24298        bncqlb    en4hp28   
2              29117         15415        bncqlb    en4kqq9   
3              29117         14389        bncqlb    en4f7vh   
4              29117          5357        bncqlb    en4m0gm   
5              29117         16081        bncqlb    en4hvq1   
6              29117         25806        bncqlb    en4n6hx   
7              29117          4892        bncqlb    en4izto   
8              29117         12290        bncqlb    en4hijc   
9              29117         17502        bncqlb    en4lisi   
10             29117         19126        bncqlb    en4et42   
11             29117         16508        bncqlb    en4je3s   
12             29117          4241        bncqlb    en4fcs2   
13             29117          6952        bncqlb    en4ip4o   
14             29117          2167        bncqlb    en4

[944 rows x 6 columns]


In [5]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at C:\Users\pedalo\.cache\torch\pytorch_pretrained_bert\26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [6]:
# Tokenized input
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)
print(tokenized_text)

['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', 'henson', 'was', 'a', 'puppet', '##eer', '[SEP]']


In [7]:
# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 8
tokenized_text[masked_index] = '[MASK]'
assert tokenized_text == ['[CLS]', 'who', 'was', 'jim', 'henson', '?', '[SEP]', 'jim', '[MASK]', 'was', 'a', 'puppet', '##eer', '[SEP]']

In [8]:
# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
print(tokens_tensor)
print(segments_tensors)

tensor([[  101,  2040,  2001,  3958, 27227,  1029,   102,  3958,   103,  2001,
          1037, 13997, 11510,   102]])
tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]])


In [9]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

# If you have a GPU, put everything on cuda
if torch.cuda.is_available() == True:
    tokens_tensor = tokens_tensor.to('cuda')
    segments_tensors = segments_tensors.to('cuda')
    model.to('cuda')

# Predict hidden states features for each layer
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)
# We have a hidden states for each of the 12 layers in model bert-base-uncased
assert len(encoded_layers) == 12

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at C:\Users\pedalo\.cache\torch\pytorch_pretrained_bert\9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file C:\Users\pedalo\.cache\torch\pytorch_pretrained_bert\9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir C:\Users\pedalo\AppData\Local\Temp\tmpdz3jazde
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size":

In [44]:
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
print(model)
model.eval()

# If you have a GPU, put everything on cuda
if torch.cuda.is_available() == True:
    tokens_tensor = tokens_tensor.to('cuda')
    segments_tensors = segments_tensors.to('cuda')
    model.to('cuda')

# Predict all tokens
with torch.no_grad():
    predictions = model(tokens_tensor, segments_tensors)
    print(predictions.size())
# confirm we were able to predict 'henson'
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
assert predicted_token == 'henson'
print(predicted_index)
print(predicted_token)

for (name, param) in model.named_parameters():
    if param.requires_grad:
        print('name', name#, 'weigth data', param.data
             )

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at C:\Users\pedalo\.cache\torch\pytorch_pretrained_bert\9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file C:\Users\pedalo\.cache\torch\pytorch_pretrained_bert\9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir C:\Users\pedalo\AppData\Local\Temp\tmp0pphl80j
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size":

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertInterm

)
torch.Size([1, 14, 30522])
27227
henson
name bert.embeddings.word_embeddings.weight
name bert.embeddings.position_embeddings.weight
name bert.embeddings.token_type_embeddings.weight
name bert.embeddings.LayerNorm.weight
name bert.embeddings.LayerNorm.bias
name bert.encoder.layer.0.attention.self.query.weight
name bert.encoder.layer.0.attention.self.query.bias
name bert.encoder.layer.0.attention.self.key.weight
name bert.encoder.layer.0.attention.self.key.bias
name bert.encoder.layer.0.attention.self.value.weight
name bert.encoder.layer.0.attention.self.value.bias
name bert.encoder.layer.0.attention.output.dense.weight
name bert.encoder.layer.0.attention.output.dense.bias
name bert.encoder.layer.0.attention.output.LayerNorm.weight
name bert.encoder.layer.0.attention.output.LayerNorm.bias
name bert.encoder.layer.0.intermediate.dense.weight
name bert.encoder.layer.0.intermediate.dense.bias
name bert.encoder.layer.0.output.dense.weight
name bert.encoder.layer.0.output.dense.bias
name ber

In [42]:
model = torchvision.models.vgg19(pretrained=True)
print(model)
for param in model.parameters():
    param.requires_grad = False
    # Replace the last fully-connected layer
    # Parameters of newly constructed modules have requires_grad=True by default
model.fc = nn.Linear(512, 8) # assuming that the fc7 layer has 512 neurons, otherwise change it 
# model.fc
# model.cuda()
# model.classifier[6] = nn.Linear(512, 8)
# model.fc
for (name, param) in model.named_parameters():
    if param.requires_grad:
        print('name', name, 'weigth data', param.data)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)